## Load Embeddings
This is an auxiliar model to load the info to Pinecone

In [1]:
import os
import openai
from streamlit_extras.add_vertical_space import add_vertical_space
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain_pinecone import Pinecone
from dotenv import load_dotenv

load_dotenv('.env')

index_name = 'thesis-model-1'
openai.api_base = os.getenv('OPENAI_ENDPOINT')
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_version = "2023-09-15-preview"
llm_model = 'gpt-35-turbo-jdrios'
emb_model = 'text-embedding-ada-002-jdrios'

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader([
    "https://www.leal.co/usuarios",
    "https://www.leal.co/puntos",
    "https://www.leal.co/leal-coins"
])
documents = loader.load()

In [3]:
documents[1].metadata

{'source': 'https://www.leal.co/puntos',
 'title': 'Tus puntos se convierten en premios',
 'description': 'Tenemos cientos de comercios donde ganas premios por hacer lo de siempre: disfrutar en tus marcas favoritas.',
 'language': 'No language found.'}

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

In [5]:
documents[0]

Document(page_content='Usuarios Leal, puntos y Leal Coins', metadata={'source': 'https://www.leal.co/usuarios', 'title': 'Usuarios Leal, puntos y Leal Coins', 'description': 'Los puntos puedes redimirlos por premios exclusivos en la marca en la que acumulas, los Leal Coins son universales y puedes usarlos en miles de marcas físicas u online que tenemos para ti.', 'language': 'No language found.'})

In [6]:
# # Computing Embeddings (with Hughing Face)
# emb_model = "sentence-transformers/all-mpnet-base-v2"
# embeddings = HuggingFaceHubEmbeddings(repo_id=emb_model,
#                                         huggingfacehub_api_token=os.getenv('HUGGINGFACEHUB_API_TOKEN'))

# # Store the embeddings

# vector_store = PineconeVectorStore.from_documents(documents, embeddings, index_name=index_name)

In [9]:
# Computing Embeddings (with OpenAI).OpenAI embeddings has 1536 dimensions
from langchain_openai import AzureOpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=emb_model,
    azure_endpoint=os.getenv('OPENAI_ENDPOINT'),
)

# Create Vector Store
vector_store = PineconeVectorStore.from_documents(documents, embeddings, index_name=index_name)

In [10]:
query = "Cual es la diferencia entre coins y puntos leal?"
docs = vector_store.similarity_search(query=query, k=3)
docs
        # st.write(docs)  # write nearest chunks
        # llm_model = OpenAI(temperature=0.9, model_name='gpt-3.5-turbo')
        # chain = load_qa_chain(llm=llm_model, chain_type="stuff")
        # with get_openai_callback() as cb:
        #     response = chain.run(input_documents=docs, question=query)
        #     # TODO: Modify prompts, and review current prompts

        #     # To know costs
        #     print(cb)
        # st.write(response)

[Document(page_content='Usuarios Leal, puntos y Leal Coins', metadata={'description': 'Los puntos puedes redimirlos por premios exclusivos en la marca en la que acumulas, los Leal Coins son universales y puedes usarlos en miles de marcas físicas u online que tenemos para ti.', 'language': 'No language found.', 'source': 'https://www.leal.co/usuarios', 'title': 'Usuarios Leal, puntos y Leal Coins'}),
 Document(page_content='tu app Leal y entérate cuantos puntos te faltan para reclamar tu premio.¿Cuál es la diferencia entre puntos y Leal Coins?Los puntos puedes redimirlos por premios exclusivos en la marca en la que acumulas, los Leal Coins son universales y puedes usarlos en miles de marcas físicas u online que tenemos para ti.¿Dónde puedo usar mis Leal Coins?Usa tus Leal Coins para adquirir bonos de marcas como Mc Donalds, Crepes & Waffles o visita nuestros comercios Leal como Texaco, Buffalo Wings y úsalos para pagar. ¡Encuentra en nuestra app todas las opciones para usarlos!Actualiza